Load Libraries

In [1]:
import tensorflow as tf
from sklearn.metrics import confusion_matrix,accuracy_score
from pprint import pprint
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import json
from random import randint
%matplotlib inline
sess = tf.InteractiveSession()
    

TensorFlow MLP Class

In [41]:
class TfAnn(object):
    
    def __init__(self):
        self.hidden=[]
        self.np_hidden=[]
        
        self.n_nodes=[]
        self.n_classes = 0
        self.n_hiden_layers = 0

    # create empty network for training
    def init_empty(self,layers,n_classes,size):
        self.n_classes = n_classes
        self.n_hiden_layers = len(layers)
        for i in range(self.n_hiden_layers):
            self.hidden.append({'weights':[],'biases':[]})
            self.np_hidden.append({'weights':[],'biases':[]})
            self.n_nodes.append(layers[i])
        self.output_layer = {'weights':[],'biases':[]}
        self.np_output_layer={"weights":[],"biases":[]}

        for i in range(self.n_hiden_layers):
            self.hidden[i] = {'weights':tf.Variable(tf.random_normal([size, self.n_nodes[i]])),
                      'biases':tf.Variable(tf.random_normal([self.n_nodes[i]]))}
            
        self.output_layer = {'weights':tf.Variable(tf.random_normal([self.n_nodes[-1], self.n_classes])),
                        'biases':tf.Variable(tf.random_normal([self.n_classes]))}

    
    def get_number_of_nodes(self,layer):
        return len(self.hidden[layer]['weights'][0])
    
    # Reinflate network from json description
    def init_json(self,jfile):
        with open(jfile) as json_data:
            tf_data = json.load(json_data)
        self.n_classes = tf_data["n_classes"]
        self.n_hiden_layers = tf_data["n_hiden_layers"]
        self.hidden =tf_data["hidden"]
        
        self.output_layer =tf_data["output"]


        
    def create(self,data,mask):
        # This is the heart of the ann where multiply the data by the wights to the layers 
        for i in range(self.n_hiden_layers):
            project_weights = [list(np.array(a)*b) for a,b in zip(self.hidden[i]['weights'],mask[i])]
            project_biases = [a*b for a,b in zip(self.hidden[i]['biases'],mask[i])]
            layer = tf.add(
                tf.matmul(data,project_weights)
                , project_biases)
            layer= tf.nn.relu(layer)
        output =  tf.add(tf.matmul(layer,self.output_layer['weights']) , self.output_layer['biases'])
        return output
    
    
    #Save a trained network as a json file
    def extract(self,jfile):
        for i in range(self.n_hiden_layers):
            self.np_hidden[i]["weights"] = neural_network_model.hidden[i]["weights"].eval().tolist()
            self.np_hidden[i]["biases"] = neural_network_model.hidden[i]["biases"].eval().tolist()
        self.np_output_layer["weights"] = neural_network_model.output_layer["weights"].eval().tolist()
        self.np_output_layer["biases"] = neural_network_model.output_layer["biases"].eval().tolist()
        with open(jfile,"w") as jout:
            json.dump({"n_classes":self.n_classes, # number of input classifier classes
                       "n_hiden_layers":self.n_hiden_layers, # number of 
                       "hidden":self.np_hidden,# weights and biases
                       # each layer is defined by dict {'weights':[],'biases':[]}
                       "output":self.np_output_layer} # as for hidden
                      ,jout)
        
        

In [42]:
mask1=[1,2,3,4]
mask2=[14,13,12,11]
cross=randint(1,len(mask1))
mask3=mask1[:cross]+mask2[cross:]
print(mask3)

[1, 2, 3, 4]


In [43]:
x = 0
x+=1
print(x)
x=x%2
print(x)
x+=1
x=x%2
print(x)
x+=1
x=x%2
print(x)
x+=1
x=x%2
print(x)
x+=1
x=x%2
print(x)

1
1
0
1
0
1


In [44]:
class Gene(object):
    size=0
    mutation_rate=0.0
    
    def __init__(self):
        self.mask=[0 for _ in range(Gene.size)]
        self.fittness=0
        
    def random(self):
        for i in range(Gene.size):
            self.mask[i] = randint(0, 1)
    
    def cross(self,gene1):
        cross_point = randint(1,Gene.size)
        gene3 = Gene()
        gene4 = Gene()
        gene3.mask=gene1.mask[:cross]+self.mask[cross:]
        gene4.mask=gene1.mask[cross:]+self.mask[:cross]
        return gene3, gene4
    
    def mutate():
        if ranom()<mutation_rate:
            mut_point =  randint(0,Gene.size)
            self.mask[mut_point]+=1
            self.mask[mut_point]=self.mask[mut_point]%2
    

In [45]:
def test_neural_network(neural_network_model,mask,x_data_test,y_data_test):
    # set up network
    x = tf.placeholder('float')
    prediction = neural_network_model.create(x,mask)    
    
    # ren test data
    y_test_res=(sess.run(prediction,feed_dict={x:x_data_test}))       
    # the correct data
    true_class=np.argmax(y_data_test,1)
    
    # get the index of the outpt array with heighest value
    predicted_class=np.argmax(y_test_res,1)
    
    # calculate confusion matix
    cm = confusion_matrix(true_class,predicted_class)
    cm = cm.astype('float')*10000 / cm.sum(axis=1)[:, np.newaxis]
    cm = np.nan_to_num(cm, copy=True)
    cm = cm.astype('int')
    return accuracy_score(true_class, predicted_class) , cm*0.01

In [47]:
with open("datasets/wine/wine_train.json") as json_data:
    test_dataset = json.load(json_data)

test_x = np.asarray(test_dataset["attribs"])
test_y = np.asarray(test_dataset["target_hot"])

neural_network_model = TfAnn()
neural_network_model.init_json("classifiers/wine-mlp.json")
Gene.size = neural_network_model.get_number_of_nodes(0)
for _ in range(10):
    a_gene = Gene()
    a_gene.random()
    acc,cf = test_neural_network(neural_network_model,[a_gene.mask],test_x,test_y)
    pprint(acc)
#plot_confusion_matrix(cf,[1,2,3],"Cf matrix","wine")

0.676056338028169
0.5211267605633803
0.528169014084507
0.49295774647887325
0.4014084507042254
0.5
0.5774647887323944
0.28169014084507044
0.5985915492957746
0.4647887323943662
